# Convolutional Neural Networks

In [1]:
import sys
import time
from os import getcwd
from os.path import join, dirname

import pandas as pd
import tensorflow as tf

sys.path.append(join(dirname(getcwd()), "src"))
from utils import (read_text_files_and_labels_with_vocab_processor,
                   loss, training_adam, training_gradient_descent,
                   evaluation, fill_feed_dict, do_eval_cnn,
                   fully_connected_network, conv_layer)

In [2]:
base_data_path = join(dirname(getcwd()), "data")
training_data_path = join(base_data_path, "test_data_revised",
                          "PRAXIS_rapid_eval_MODEL_TRAINING_2015/*/*.txt")
test_data_path = join(base_data_path, "test_data_revised",
                      "PRAXIS_rapid_eval_TESTING_2015/*/*.txt")

In [3]:
train_labels_path = join(base_data_path, "test_data_revised",
                         "training_macro.csv")
test_labels_path = join(base_data_path, "test_data_revised",
                        "testing_macro.csv")
df = pd.read_csv(train_labels_path)
df = pd.concat([df, pd.read_csv(test_labels_path)])
df = df[["appointment_id", "H1"]]
df.rename(columns={"appointment_id": "id", "H1": "label"}, inplace=True)
ids_to_labels_dict = {}
if len(df.id) != len(set(df.id)):
    raise ValueError("Duplicate IDs!")
for id_ in df.id:
    ids_to_labels_dict[id_] = df[df.id == id_].iloc[0].label

In [4]:
# Create DataSet objects by using read_text_files_and_labels
(train_data, test_data, dev_data) = \
    read_text_files_and_labels_with_vocab_processor(ids_to_labels_dict,
                                                    training_data_path,
                                                    test_data_path,
                                                    get_id_from_text_file_func=
                                                        lambda x: int(x[:16]))

In [5]:
# show_data = False
show_data = True

In [6]:
if show_data:
    print("Shape of data:\n\tTraining: {}\n\tTest: {}"
          .format(train_data._features.shape,
                  test_data._features.shape))

Shape of data:
	Training: (4000, 1255)
	Test: (2750, 1255)


In [7]:
train_data._features

array([[   1,    2,    3, ...,    0,    0,    0],
       [   1,   89,  148, ...,    0,    0,    0],
       [  19,  203,  244, ...,    0,    0,    0],
       ..., 
       [  53,  942,  340, ...,    0,    0,    0],
       [  24,  157,   43, ...,    0,    0,    0],
       [ 456,   89, 3212, ...,    0,    0,    0]], dtype=int32)

In [8]:
# Define some parameters
log_dir_path = join(getcwd(), "logs")
max_steps = 10000
optimizer_type = "adam"
#optimizer_type = "gradient descent"
learning_rate = 0.01
hidden1 = 512
hidden2 = 128
hidden3 = 16
# Use shape of `train_data._features` since it should be the same as for
# `test_data._features`/`dev_data._features`
NUM_FEATURES = train_data._features.shape[1]
batch_size = 10
NUM_CLASSES = 6
dropout = 0.5
NUM_FILTERS = 20
FILTERS = 3

In [9]:
# Tell TensorFlow that the model will be built into the default Graph.
with tf.Graph().as_default():

    # Generate placeholders for the input feature data and labels.
    inputs_placeholder = tf.placeholder(tf.float32, shape=(batch_size,
                                                           NUM_FEATURES))
    labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))
    
    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
    
    x = tf.reshape(inputs_placeholder, shape=[-1, 1, NUM_FEATURES, 1])

    conv1 = conv_layer(x, 2, NUM_FILTERS, 2, 2)
    print(conv1.get_shape())
    conv2 = conv_layer(x, 3, NUM_FILTERS, 2, 2)
    print(conv2.get_shape())
    conv3 = conv_layer(x, 4, NUM_FILTERS, 2, 2)
    print(conv3.get_shape())
    
    conv = tf.concat(2, [conv1, conv2, conv3])
    print(conv.get_shape())
    
    reshape_length = NUM_FILTERS*FILTERS*conv1.get_shape().as_list()[2]
    input_fc = tf.reshape(conv, [-1, reshape_length])
    
    logits = fully_connected_network(input_fc, [reshape_length, 14, 14], keep_prob, NUM_CLASSES)
    
    # Add to the Graph the Ops for loss calculation.
    loss_ = loss(logits, labels_placeholder)
    
    # Add to the Graph the Ops that calculate and apply gradients.
    if optimizer_type == "adam":
        train_op = training_adam(loss_, learning_rate)
    elif optimizer_type == "gradient descent":
        train_op = training_gradient_descent(loss_, learning_rate)
    else:
        raise ValueError("Choose either \"adam\" or \"gradient descent\" for "
                         "`optimizer_type`.")

    # Add the Op to compare the logits to the labels during evaluation.
    eval_correct = evaluation(logits, labels_placeholder)

    # Build the summary Tensor based on the TF collection of Summaries.
    summary = tf.summary.merge_all()

    # Add the variable initializer Op.
    init = tf.global_variables_initializer()

    # Create a saver for writing training checkpoints.
    saver = tf.train.Saver()

    # Create a session for running Ops on the Graph.
    sess = tf.Session()

    # Instantiate a SummaryWriter to output summaries and the Graph.
    summary_writer = tf.summary.FileWriter(log_dir_path, sess.graph)

    # And then after everything is built:

    # Run the Op to initialize the variables.
    sess.run(init)

    # Start the training loop.
    for step in range(max_steps):
        start_time = time.time()

        # Fill a feed dictionary with the actual set of images and labels
        # for this particular training step.
        feed_dict = fill_feed_dict(train_data,
                                   inputs_placeholder,
                                   labels_placeholder,
                                   batch_size)

        feed_dict[keep_prob] = dropout
        # Run one step of the model.  The return values are the activations
        # from the `train_op` (which is discarded) and the `loss` Op.  To
        # inspect the values of your Ops or variables, you may include them
        # in the list passed to sess.run() and the value tensors will be
        # returned in the tuple from the call.
        _, loss_value = sess.run([train_op, loss_],
                                 feed_dict=feed_dict)

        duration = time.time() - start_time

        # Write the summaries and print an overview fairly often.
        if step % 100 == 0:

            # Print status to stdout.
            print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
            # Update the events file.
            summary_str = sess.run(summary, feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, step)
            summary_writer.flush()
        
        # Save a checkpoint and evaluate the model periodically.
        if (step + 1) % 1000 == 0 or (step + 1) == max_steps:
            checkpoint_file = join(log_dir_path, 'model.ckpt')
            saver.save(sess, checkpoint_file, global_step=step)

            # Evaluate against the training set.
            print('Train Data Eval:')
            do_eval_cnn(sess,
                        eval_correct,
                        inputs_placeholder,
                        labels_placeholder,
                        train_data,
                        logits,
                        batch_size,
                        keep_prob, dropout)

            # Evaluate against the test set.
            print('Test Data Eval:')
            do_eval_cnn(sess,
                        eval_correct,
                        inputs_placeholder,
                        labels_placeholder,
                        test_data,
                        logits,
                        batch_size,
                        keep_prob, dropout)

(10, 1, 627, 20)
(10, 1, 627, 20)
(10, 1, 627, 20)
(10, 1, 1881, 20)
Step 0: loss = 283788.19 (0.022 sec)
Step 100: loss = 1.17 (0.007 sec)
Step 200: loss = 0.91 (0.008 sec)
Step 300: loss = 1.23 (0.009 sec)
Step 400: loss = 1.18 (0.007 sec)
Step 500: loss = 1.29 (0.009 sec)
Step 600: loss = 0.92 (0.008 sec)
Step 700: loss = 1.45 (0.008 sec)
Step 800: loss = 1.19 (0.007 sec)
Step 900: loss = 1.32 (0.008 sec)
Train Data Eval:
  Num examples: 4000  Num correct: 2086  Accuracy @ 1: 0.5215
Test Data Eval:
  Num examples: 2750  Num correct: 1394  Accuracy @ 1: 0.5069
Step 1000: loss = 0.91 (0.008 sec)
Step 1100: loss = 1.35 (0.008 sec)
Step 1200: loss = 1.19 (0.010 sec)
Step 1300: loss = 1.32 (0.009 sec)
Step 1400: loss = 0.90 (0.008 sec)
Step 1500: loss = 1.35 (0.009 sec)
Step 1600: loss = 1.19 (0.008 sec)
Step 1700: loss = 1.31 (0.008 sec)
Step 1800: loss = 0.90 (0.008 sec)
Step 1900: loss = 1.35 (0.010 sec)
Train Data Eval:
  Num examples: 4000  Num correct: 2086  Accuracy @ 1: 0.5215
Te